We will try to get the information of all Pokémons (Name,id,attribute,etc...) from: https://pokemondb.net/pokedex/national
using web scraping techniques.

In [80]:
import requests # To be able to access websites we wish to scrap.
from bs4 import BeautifulSoup
import csv
import pandas as pd
import time
from IPython.display import Image, HTML

In [2]:
url='https://pokemondb.net/pokedex/national'

In [3]:
req=requests.get(url) 
status=req.status_code # Status of the connection attempt if everything is good this should have a 200 value.
encoding=req.encoding
text=BeautifulSoup(req.text,'html.parser')
print(status)

200


In [24]:
data=text.findAll('span',{'class':'infocard-lg-data'})
for item in data:
    pass
item

<span class="infocard-lg-data text-muted"><small>#809</small><br/><a class="ent-name" href="/pokedex/melmetal">Melmetal</a><br/><small><a class="itype steel" href="/type/steel">Steel</a></small></span>

In [25]:
imgs=text.findAll('span',{'class':'infocard-lg-img'})
for img in imgs:
    pass
img

<span class="infocard-lg-img"><a href="/pokedex/melmetal"><span class="img-fixed img-sprite" data-alt="Melmetal icon" data-src="https://img.pokemondb.net/sprites/ultra-sun-ultra-moon/small/melmetal.jpg"></span></a></span>

In [34]:
PokId=item.find('small').text
PokName=item.find('a',{'class':'ent-name'}).text
PokAttrib=item.find('a',{'class':'itype'}).text
PokImg=img.find('a').find('span',{'class':'img-fixed'}).get('data-src')

'https://img.pokemondb.net/sprites/ultra-sun-ultra-moon/small/melmetal.jpg'

In [47]:
def get_img(url_img,id):
    r=requests.get(url_img)
    if r.status_code==200:
        #img = Image.open(BytesIO(r.content))
        img_container="C:/Users/YsfEss/Desktop/Pokemon/"+str(id)+'.jpg'
        with open(img_container ,'wb') as f:
            f.write(r.content) 
            print('Image saved!')
    else:
        print('Connection trouble!')

In [41]:
# Creating database of Pokemons
data=text.findAll('span',{'class':'infocard-lg-data'})
PokeDB=pd.DataFrame(columns=['Id','Name','Attribute'])
i=0
for item in data:
    PokId=item.find('small').text
    PokName=item.find('a',{'class':'ent-name'}).text
    PokAttrib=item.find('a',{'class':'itype'}).text
    temp=[PokId,PokName,PokAttrib]
    PokeDB.loc[i]=temp  
    i+=1
pd.DataFrame.to_csv(PokeDB,'PokeDB.csv')
PokeDB.head()

,Id,Name,Attribute
0,#001,Bulbasaur,Grass
1,#002,Ivysaur,Grass
2,#003,Venusaur,Grass
3,#004,Charmander,Fire
4,#005,Charmeleon,Fire


In [ ]:
# Saving images:
i=1
for img in imgs:
    PokImg=img.find('a').find('span',{'class':'img-fixed'}).get('data-src')
    get_img(PokImg,i)
    i+=1
    if i%10==0:
        time.sleep(2) # To not be agressive

Now we will extract info about Pokédex data, Training & Breeding into 3 tables. 
The choosen pokemon will be: Gengar https://pokemondb.net/pokedex/gengar


In [4]:
url='https://pokemondb.net/pokedex/gengar'

In [5]:
req=requests.get(url) 
status=req.status_code # Status of the connection attempt if everything is good this should have a 200 value.
encoding=req.encoding
text=BeautifulSoup(req.text,'html.parser')
print(status)

200


In [6]:
data=text.find_all('table',{'class':'vitals-table'})
tables=[]
i=0
for item in data:
    if i<3:
        tables.append(item)
    i+=1
Pokedex=tables[0]
Training=tables[1]
Breeding=tables[2]

In [92]:
Pokedex.find_all('tr')[6]

<tr>
<th>Local №</th>
<td>094 <small class="text-muted">(Red/Blue/Yellow)</small><br/>060 <small class="text-muted">(Gold/Silver/Crystal)</small><br/>094 <small class="text-muted">(FireRed/LeafGreen)</small><br/>071 <small class="text-muted">(Diamond/Pearl)</small><br/>071 <small class="text-muted">(Platinum)</small><br/>060 <small class="text-muted">(HeartGold/SoulSilver)</small><br/>032 <small class="text-muted">(X/Y — Mountain Kalos)</small><br/>063 <small class="text-muted">(Sun/Moon — Alola dex)</small><br/>075 <small class="text-muted">(U.Sun/U.Moon — Alola dex)</small><br/>094 <small class="text-muted">(Let's Go Pikachu/Let's Go Eevee)</small><br/></td>
</tr>

In [98]:
# Pokedex:
TPokedex=pd.DataFrame(columns=['National N','Type','Power','Species','Height','Weight','Abilities','Local N'])
K=Pokedex.find_all('tr')
NatNum=K[0].find('strong').text
Type=K[1].find_all('a')[0].text
Pow=K[1].find_all('a')[1].text
Spec=K[2].find('td').text
Height=K[3].find('td').text
Weight=K[4].find('td').text
Abilities=K[5].find('a').text

# Cleaning the list form useless tags
cleanL=[]
d=Pokedex.find_all('tr')[6].find_all('td')[0].contents
for i in range (len(d)):
    if str(d[i])!='<br/>':
        cleanL.append(d[i])

LocalNum=[]
for i in range (0,len(cleanL)-1,2):
    LocalNum.append([cleanL[i],cleanL[i+1].text])
LocalNum 


TPokedex.loc[0]=[NatNum,Type,Pow,Spec,Height,Weight,Abilities,"\n".join([':'.join(item) for item in LocalNum])]
pd.set_option('display.max_colwidth', -1)
HTML(TPokedex.to_html(escape=False).replace("\\n","<br>"))

,National N,Type,Power,Species,Height,Weight,Abilities,Local N
0,094,Ghost,Poison,Shadow Pokémon,1.5 m (4′11″),40.5 kg (89.3 lbs),Cursed Body,094 :(Red/Blue/Yellow)060 :(Gold/Silver/Crystal)094 :(FireRed/LeafGreen)071 :(Diamond/Pearl)071 :(Platinum)060 :(HeartGold/SoulSilver)032 :(X/Y — Mountain Kalos)063 :(Sun/Moon — Alola dex)075 :(U.Sun/U.Moon — Alola dex)094 :(Let's Go Pikachu/Let's Go Eevee)


In [ ]:
# Training:

Training=pd.DataFrame(columns=['EV yield','Catch rate','Base Friendship','Base Exp.','Growth Rate'])
K=Pokedex.find_all('tr')
EVyield=K[0].find('strong').text
Type=K[1].find_all('a')[0].text
Pow=K[1].find_all('a')[1].text
Spec=K[2].find('td').text
Height=K[3].find('td').text
Weight=K[4].find('td').text
Abilities=K[5].find('a').text
